# TEsting fuzzywuzzy

In [2]:

from datasets import load_dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer


# Example: Load your dataset with tex col
dataset = load_dataset("imdb")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

def convert_labels(example):
    example['labels'] = example['label']  # Assuming your label column is 'label'
    return example

tokenized_datasets = tokenized_datasets.map(convert_labels)

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

# find total number of classes
num_labels = len(set(train_dataset['labels']))

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

In [34]:
import pickle as pkl
import pandas as pd

with open("../data/induced_errors_v1.pkl", "rb") as f:
    df = pkl.load(f)

In [18]:
# remove the col called "old_text"
df = df.drop(columns=["old_text"])

In [38]:
# filter rows with distance 1.0 and error_name "transposition"
df[(df["distance"] == 1.0)]["error_name"].value_counts()

error_name
    12665
Name: count, dtype: int64

# Testing Flava

In [12]:
import torch
from transformers import FlavaProcessor, FlavaModel
from PIL import Image
import urllib.request
from PIL import Image


# Load pre-trained model and processor
processor = FlavaProcessor.from_pretrained("facebook/flava-full")
model = FlavaModel.from_pretrained("facebook/flava-full")

# Prepare inputs
text = "GG"
url = "https://media.geeksforgeeks.org/wp-content/uploads/20210224040124/JSBinCollaborativeJavaScriptDebugging6-300x160.png" 
urllib.request.urlretrieve(url, "geeksforgeeks.png")
image = Image.open(r"geeksforgeeks.png").convert('RGB')


inputs = processor(text=[text], images=image, return_tensors="pt", max_length=128, padding="max_length", truncation=True)

# Generate embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Access different types of embeddings
text_embeddings = outputs.text_embeddings
image_embeddings = outputs.image_embeddings
multimodal_embeddings = outputs.multimodal_embeddings

print(f"Text Embedding Shape: {text_embeddings.shape}")
print(f"Image Embedding Shape: {image_embeddings.shape}")
print(f"Multimodal Embedding Shape: {multimodal_embeddings.shape}")

/mnt/workspace1/ErrorClass/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Text Embedding Shape: torch.Size([1, 128, 768])
Image Embedding Shape: torch.Size([1, 197, 768])
Multimodal Embedding Shape: torch.Size([1, 326, 768])


/mnt/workspace1/ErrorClass/lib/python3.12/site-packages/transformers/modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [13]:
text_embeddings.shape, image_embeddings.shape, multimodal_embeddings.shape, outputs.keys()

(torch.Size([1, 128, 768]),
 torch.Size([1, 197, 768]),
 torch.Size([1, 326, 768]),
 odict_keys(['image_embeddings', 'image_output', 'text_embeddings', 'text_output', 'multimodal_embeddings', 'multimodal_output']))

# Testing PHI

In [ ]:
!pip install flash-attn transformers torch torchvision

In [8]:
from transformers import AutoProcessor, AutoModelForCausalLM # Load AutoModelForCausalLM instead of AutoModelForVision2Seq
from PIL import Image
import torch
import urllib.request

# Load model and processor
processor = AutoProcessor.from_pretrained("microsoft/Phi-3-vision-128k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-vision-128k-instruct",trust_remote_code=True) # Load the model using AutoModelForCausalLM

# Function to perform OCR on a single image
def perform_ocr(image_path, prompt="Perform OCR on this image:"):
    # Load the image
    image = Image.open(image_path)
    
    # Create the input text with an image tag
    input_text = f"{prompt} <image>"
    
    # Process the inputs
    inputs = processor(text=input_text, images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
    
    text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return text

# Example usage
url = "https://static.startuptalky.com/2021/06/GeeksforGeeks-StartupTalky.jpg"
urllib.request.urlretrieve(url, "moco.png")
extracted_text = perform_ocr("moco.png")
print(f"Extracted text: {extracted_text}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


AssertionError: total images must be the same as the number of image tags, got 0 image tags and 1 images

In [ ]:
!df -h